# problem


create data set randomly

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

num_customers = 20

customer_coords = np.random.rand(num_customers, 2) * 100

depot_coords = np.array([[50, 50]])

all_coords = np.vstack([depot_coords, customer_coords])

coords_df = pd.DataFrame(all_coords, columns=['x', 'y'])

coords_df.to_csv('vrp_data.csv', index=False)

print(coords_df)

            x          y
0   50.000000  50.000000
1    3.872700  67.117508
2   48.504976  65.015717
3   49.000423  79.368573
4   65.897282   0.351010
5   39.978548  24.425447
6   29.389174  35.679227
7   27.907541  34.957806
8   84.930842  20.858977
9   50.741262  38.991553
10  99.159049  99.886426
11  85.154339  58.864615
12  64.500273  50.089549
13  77.113278   1.176729
14  83.744807  17.668559
15  71.955880  94.882633
16  32.304743  52.457533
17  37.203235   7.559780
18  93.880158  62.315133
19  96.271284  33.835365
20  55.356449  97.492279


# encoding

In [236]:
import numpy as np
import pandas as pd

coords_df = pd.read_csv('vrp_data.csv')

all_coords = coords_df.values

customer_indices = np.arange(0, len(all_coords))

customer_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [237]:
import plotly.express as px

fig = px.scatter(coords_df, x='x', y='y', color=['Depot'] + ['Customer']*num_customers, 
                 labels={'color': 'Location Type'}, title='Customer and Depot Locations')

fig.show()


# population intilaztion

In [238]:
def initialize_population(pop_size, num_customers):
    customer_indices = np.arange(1, num_customers + 1)  # Exclude depot
    population = []
    for _ in range(pop_size):
        route = np.random.permutation(customer_indices)  # Random customer order
        route = np.concatenate(([0], route, [0]))       # Add depot at start and end
        population.append(route)

    return np.array(population)


# huristic function

In [239]:
def calculate_distance(route, coords):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += np.linalg.norm(coords[route[i]] - coords[route[i + 1]])
    return total_distance


# selection function

In [240]:
import random

def selection(population, fitness_scores, num_parents):
    sorted_indices = np.argsort(fitness_scores)
    ranked_population = [population[i] for i in sorted_indices]
    selected_parents = ranked_population[:num_parents]
    return selected_parents

# crossover function

In [241]:
def crossover(parent1, parent2):
    size = len(parent1) - 2  

    point = np.random.randint(1, size)  

    child = [-1] * len(parent1) 
    child[0], child[-1] = 0, 0  
    child[1:point + 1] = parent1[1:point + 1]  
    pointer = point + 1
    
    for gene in parent2:
        if gene not in child:
            child[pointer] = gene
            pointer += 1
    return np.array(child)


In [242]:
def mutate(route, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(range(1, len(route) - 1), 2, replace=False)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route


<h1>evlolve</h1>
<p>
    <!-- <img src="spinning-cat.gif" alt="Spinning Cat"> -->
</p>

<style>
    h1 {
        color: red;
    }
    p {
        position: relative;
        /* animation: moveRightLeft 2s infinite ease-in-out; */
    }
    img
    {
        width :100px;
        height :100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [243]:
def validate_and_repair(route, num_customers):
    all_customers = set(range(1, num_customers + 1))
    visited = set(route[1:-1])  # Exclude depot
    missing_customers = list(all_customers - visited)
    duplicates = [c for c in route[1:-1] if route[1:-1].tolist().count(c) > 1]

    for i in range(1, len(route) - 1):
        if route[i] in duplicates:
            route[i] = missing_customers.pop()
            duplicates.remove(route[i])
    return route


In [244]:
def evolve_population(population, coords, num_customers, mutation_rate, retain_rate):
    fitness_scores = [1 / calculate_distance(route, coords) for route in population]

    sorted_indices = np.argsort(fitness_scores)[::-1]
    retain_length = int(len(population) * retain_rate)
    parents = [population[i] for i in sorted_indices[:retain_length]]

    children = []
    while len(children) < len(population) - len(parents):

        p1, p2 = parents[0], parents[1]  # Just use the first two parents

        child = crossover(p1, p2)
        children.append(child)


    next_gen = parents + children
    next_gen = [validate_and_repair(mutate(route, mutation_rate), num_customers) for route in next_gen]

    return np.array(next_gen)


# trying the alogrithm 

In [245]:
import numpy as np

def run_ga(coords, num_customers, population_size=100, generations=1000, mutation_rate=0.02, retain_rate=0.2):
    
    population = initialize_population(population_size, num_customers)

    best_route = None
    best_distance = float('inf')
    best_distance_arr = []

    for gen in range(generations):
        population = evolve_population(population, coords, num_customers, mutation_rate, retain_rate)

        distances = [calculate_distance(route, coords) for route in population]
        min_distance = min(distances)

        # Update the best solution
        if min_distance < best_distance:
            best_distance = min_distance
            best_route = population[np.argmin(distances)]

        if gen % 50 == 0 or gen == generations - 1:
            print(f"Generation {gen + 1}/{generations}: Best Distance = {best_distance:.2f}")
            best_distance_arr.append(best_distance)

    best_distance_arr = np.array(best_distance_arr)
    return best_route, best_distance, best_distance_arr

In [246]:



coords = coords_df[['x', 'y']].to_numpy()

num_customers = len(coords) - 1

best_route, best_distance, best_distance_arr = run_ga(coords, num_customers)
print("maybe a best route:", best_route)
print("a maybe with a qeustion mark best distance  :", best_distance)


Generation 1/1000: Best Distance = 787.99
Generation 51/1000: Best Distance = 536.36
Generation 101/1000: Best Distance = 497.87
Generation 151/1000: Best Distance = 497.87
Generation 201/1000: Best Distance = 497.87
Generation 251/1000: Best Distance = 497.87
Generation 301/1000: Best Distance = 497.87
Generation 351/1000: Best Distance = 497.87
Generation 401/1000: Best Distance = 497.87
Generation 451/1000: Best Distance = 497.87
Generation 501/1000: Best Distance = 497.87
Generation 551/1000: Best Distance = 497.87
Generation 601/1000: Best Distance = 497.87
Generation 651/1000: Best Distance = 497.87
Generation 701/1000: Best Distance = 497.87
Generation 751/1000: Best Distance = 497.87
Generation 801/1000: Best Distance = 497.87
Generation 851/1000: Best Distance = 497.87
Generation 901/1000: Best Distance = 497.87
Generation 951/1000: Best Distance = 497.87
Generation 1000/1000: Best Distance = 497.87
maybe a best route: [ 0 16  1  7  6  9  8 14 13  4 17  5  2 12 19 11 18 10 15 

<p>
    <img src="spinning-cat.gif" alt="Spinning Cat"> 
</p>
<h1>Loading...</h1>

<style>
    h1 {
        color: red;
        position: relative;
        animation: moveRightLeft 2s infinite ease-in-out; 
    }
    p {

    }
    img {
        width: 100px;
        height: 100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [247]:
import plotly.express as px

loss_df = pd.DataFrame({'Generation': np.arange(0, len(best_distance_arr) * 50, 50), 'Best Distance': best_distance_arr})

plot = px.line(loss_df, x='Generation', y='Best Distance', title='Best Distance Over Generations')
plot.show()


In [248]:
import plotly.graph_objects as go

def plot_route(coords_df, best_route):


    route_coords = coords_df.to_numpy()
    route_x = [route_coords[i][0] for i in best_route]
    route_y = [route_coords[i][1] for i in best_route]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=route_x,
        y=route_y,
        mode='markers+lines',
        name='Route',
        marker=dict(size=10, color='blue'),
        line=dict(color='orange', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=[route_x[0], route_x[-1]],  
        y=[route_y[0], route_y[-1]],  
        mode='markers',
        name='Depot',
        marker=dict(size=12, color='red')
    ))

    fig.update_layout(
        title='Best Route Plot',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        showlegend=True
    )

    # Show plot
    fig.show()


In [249]:
plot_route(coords_df, best_route)


# differntial evolution

In [250]:
#Practical Advice: (e.g., start with NP = 10 * D, and CR = 0.9, F = 0.8)

num_customers = len(coords) - 1
num_populations = 10 * num_customers
num_populations



200

intialze population for differntial evolution

In [251]:
diff_population=initialize_population(num_populations, num_customers)
diff_population

array([[ 0, 19,  1, ..., 14, 17,  0],
       [ 0,  5,  4, ..., 15,  6,  0],
       [ 0, 17, 10, ...,  6,  4,  0],
       ...,
       [ 0,  4, 20, ..., 11,  3,  0],
       [ 0, 14, 11, ...,  6, 19,  0],
       [ 0, 14, 13, ..., 16,  2,  0]])

# mutation function for differntial 

In [252]:

def mutate(diff_population, target_idx, F):

    pop_size = len(diff_population)

    indices = list(range(pop_size))
    indices.remove(target_idx) # so i dont chooce target for calculation
    x1, x2, x3 = diff_population[np.random.choice(indices, 3, replace=False)]

    mutant = x3 + F * (x2 - x1)


    mutant = np.round(mutant).astype(int)# formula can result in float values so i round them to int

    return mutant

F = 0.8  
target_idx = 0  
mutant_vector = mutate(diff_population, target_idx, F)
print(mutant_vector)


[ 0  7 15  5 17  1 13 20 -3 25 10 22 17 -2  1 25 -3 13 10 14  4  0]


In [253]:
# mutant_vector = validate_and_repair(mutant_vector, num_customers)

In [254]:
CR = 0.9  
target = diff_population[0]  
mutant = mutate(diff_population, 0, F=0.8)  

if np.random.rand() < CR:
    child = crossover(target, mutant)  
else:
    child = target  
child 

array([ 0, 19,  1,  2,  7, 20,  5, 15,  4, 18,  6, 13, 12, 11, 10,  8, 16,
        9,  3, 14, 17,  0])